# BrkRaw Tutorial 1: Getting Started

This notebook mirrors the official Getting Started guidance using the
latest BrkRaw API and the Git LFS example datasets.


## 1. Download example data (Git LFS)

The example datasets live in `brkraw-dataset` and are stored with Git LFS.
This cell clones the repo into `data/brkraw-dataset` and pulls the LFS data.


In [ ]:
from pathlib import Path
import subprocess
import shutil

DATA_DIR = Path("data")
DATASET_REPO = DATA_DIR / "brkraw-dataset"

def ensure_git_lfs():
    if shutil.which("git") is None:
        raise RuntimeError("git is required but not found in PATH.")
    result = subprocess.run(["git", "lfs", "version"], capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError("git-lfs is required. Install it and rerun this cell.")
    return result.stdout.strip()

print(ensure_git_lfs())
DATA_DIR.mkdir(parents=True, exist_ok=True)

if not DATASET_REPO.exists():
    subprocess.run(["git", "clone", "https://github.com/BrkRaw/brkraw-dataset", str(DATASET_REPO)], check=True)

subprocess.run(["git", "-C", str(DATASET_REPO), "lfs", "pull"], check=True)
print("Dataset repo ready:", DATASET_REPO)


## 2. Load a dataset

We will use the Paravision 6.0.1 example archive.


In [ ]:
from pathlib import Path
import brkraw as brk

dataset_zip = DATASET_REPO / "PV6.0.1" / "UNC_PV6.0.1_FLASH_TurboRARE_EPI.zip"
dataset_zip.exists()


In [ ]:
loader = brk.load(str(dataset_zip))
loader.avail


## 3. Inspect study and scan metadata

BrkRaw can return info tables as dictionaries for programmatic access.


In [ ]:
info = loader.info(scope="full", as_dict=True)
info["Study"]


In [ ]:
sorted(loader.avail.keys())


## 4. Read data arrays and affines

Pick the first available scan/reco pair.


In [ ]:
import numpy as np

scan_id = sorted(loader.avail.keys())[0]
reco_id = sorted(loader.avail[scan_id].avail.keys())[0]
scan_id, reco_id


In [ ]:
def first_pack(value):
    return value[0] if isinstance(value, tuple) else value

data = first_pack(loader.get_dataobj(scan_id, reco_id=reco_id))
affine = first_pack(loader.get_affine(scan_id, reco_id=reco_id))
data.shape, affine


## 5. Quick visualization

Plot a center slice to verify the data load.


Note: These are 2D (x, y) axis plots of a single slice. We set `origin="lower"` to match the usual neuroimaging display convention so the image axes align with RAS-style orientation in this 2D view.

In [ ]:
import matplotlib.pyplot as plt

slice_idx = data.shape[2] // 2
plt.imshow(np.rot90(data[:, :, slice_idx]), cmap="gray", origin="lower")
plt.title(f"Scan {scan_id}, Reco {reco_id}, slice {slice_idx}")
plt.axis("off")


## 6. Convert to NIfTI

Convert the selected scan to NIfTI and write it to `outputs/`.


In [ ]:
from pathlib import Path

out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)

nifti = loader.convert(scan_id, reco_id=reco_id, format="nifti")
if isinstance(nifti, tuple):
    for idx, img in enumerate(nifti, start=1):
        img.to_filename(out_dir / f"scan{scan_id}_reco{reco_id}_slpack{idx}.nii.gz")
else:
    nifti.to_filename(out_dir / f"scan{scan_id}_reco{reco_id}.nii.gz")


## 7. Metadata and parameter search

Pull metadata and search for a parameter key.


In [ ]:
meta = loader.get_metadata(scan_id, reco_id=reco_id)
if meta:
    list(meta.keys())[:10]
else:
    meta


In [ ]:
loader.search_params("PVM_RepetitionTime", scan_id=scan_id)
